# Getting Started with MongoDB for Spatial Analysis

## Launch MongoDB locally using Docker

### Create a persistent data volume

To get started, you'll need to create a persistent data volume that Docker can use to store your database's files.  Creating a Docker volume allows you to persist any data loaded into the database, even if you have to stop, start, or re-run the Docker container housing your database.  Run the following from the command line to create a Docker volume for you MongoDB database:

```bash
docker volume create mongodb_volume
```

To check that the volume was created successfully, run the following from your command line:
```bash
docker volume ls
```

You should see a volume called `mongodb_volume` listed in the results.

### Launch the Docker container

Now you're ready to launch MongoDB using Docker!  Run the following from your command line:

```bash
docker run -d \
--name mongodb \
-p 27017:27017 \
--mount source=mongodb_volume,target=/data/db \
mongo:4.0
```

What does this command do?  Breaking it down, here's what each argument means:

- **-d** : runs the container in "detached" mode, so that it keeps running in the background and will not shut off if you close out of your console window
- **--name** : the name Docker will give to your container; if you don't specify a name here, Docker will give your container a randomly-generated name
- **-p** : these are port mappings, indicating that Docker should forward information going in and out of port 27017 from the Neo4j container to port 27017 on your local machine
- **--mount** : takes the persistent volume named `mongodb_volume` and mounts it into the /data folder inside of the container; this is where the database's data and settings will get stored
- **-e**: environment variables that get passed to the database's configuration file on startup; these variables are required to make sure plugins will run correctly
- At the very end of the command, you'll notice we list **mongo:4.0** as the final argument.  This specifies the Docker image and version to run inside of the container, and will download and launch MongoDB version 4.0 (the most current version as of this writing).  If newer versions are available, you can specify `mongo:latest` to get the most recent version of MongoDB.

To check that the container is running, execute the following command in the command line:

```docker container ls```

You should see something like this, indicating that the container is successfully running:

```
CONTAINER ID     IMAGE          COMMAND                  CREATED          STATUS         PORTS               
blahblahblah     mongo:4.0      "docker-entrypoint.s…"   X seconds ago    X seconds ago  0.0.0.0:27017->27017/tcp
```

<hr>

## Connect to the database

There are two ways to connect to MongoDB database once it is up and running:

#### Method #1: Mongo Compass app

MongoDB offers a desktop client software you can use to perform basic queries and other administrative tasks.  To use the client, download the **Mongo Compass app** from the [MongoDB download center](https://www.mongodb.com/download-center#compass).  Be sure to choose the version of the app that matches the operating system on your local computer.  Once that's downloaded, enter the connection credentials as follows, then click "Connect":

![MongoDB Compass connect app](img_mongodb/mongodb_compass_connect.png)

#### Method #2: mongo shell

The other way to connect to the MongoDB database is via the command line, using the **mongo shell**.  To start off, run the following command that allows you to enter the Docker container and access a bash shell within the container:

```bash
docker exec -it mongodb /bin/bash
```

Once you're inside the container with the bash prompt ready, run the following command to enter the mongo shell:

```bash
mongo --host localhost:27017
```

You'll notice that neither of these approaches requires you to authenticate or set a username or password.  This is because MongoDB runs in a kind of "demo mode" by default, where authentication features are diabled.  You will need to enable an authentication method and take other security precautions to secure the database installation if you want to use MongoDB in a production environment.

<hr>

## Load data into the database

### Create database

#### Method 1: Mongo Compass desktop app

To create a new database, click the "Create Database" button in the Mongo Compass app:

![Mongo Compass create database button](img_mongodb/mongodb_create_db.png)

Then, give the database a name, and also add a name for the first collection in the database.  A **collection** is a set of documents--in this case, tweets--that are stored together within a database.  If you are familiar with relational databases, you can think of a collection as being similar to a "table" in a relational database system.  The primary difference between a "collection" and a "table" is that collections don't enforce any kind of tabular data structure when storing your data.  Unlike rows in a table, which must be structured according to a pre-defined set of attributes/columns, documents in a collection can contain irregularly-shaped values (ex: arrays, dictionaries), missing values, and are just generally more flexible in their structure than tables generally are.

<img src="img_mongodb/mongodb_create_db_modal.png" alt="Mongo Compass create database modal" style="width: 400px;"/>


#### Method 2: mongo shell

To view a list of existing databases from within the mongo shell, run the following command:

```bash
show dbs
```

If you've already created the `twitter_sample` database using the Mongo Compass app, you should see it listed here.  Otherwise, to switch to a database--either existing or new--run the `use` command, followed by the database name.  This will activate the chosen database and/or create the database if it doesn't yet exist:

```bash
use twitter_sample
```
To create a collection within a database, run the following command:

```bash
db.createCollection("tweets")
```

### Create indexes

At this point, you'll want to create at least one index on the tweet "id" field to help detect duplicated and guarantee uniqueness when loading tweets into the database.  Once this index is set up, you don't necessarily need to create any additional indexes prior to loading the data, but later on we'll look into adding additional indexes to optimize query performance when you're ready to query the data. 

#### Method 1: Mongo Compass desktop app

To set up an index using the Compass app, navigate to the "Indexes" tab within the collection called "tweets" that you have just created.  There, you may notice that it already looks like an index named `_id_` is populated in the database.  This is the default index, and maps to the internal [ObjectId](https://docs.mongodb.com/manual/reference/method/ObjectId/) that Mongo uses when it stores each document into the collection:  

![Mongo Compass index tab with default index](img_mongodb/mongodb_index_tab.png)

You can leave this index alone, and just note that this is _not_ the same as the "id" field coming from the Twitter data.  To create an index for the "id" field within the tweets you'll be loading, click the "Create Index" button and enter the following parameters.  Then click "Create" to add the index:

<img src="img_mongodb/mongodb_create_index_modal.png" alt="Mongo Compass create index modal" style="width: 400px;"/>


#### Method 2: mongo shell

You can perform exactly the same index creation operation in the mongo shell.  Just make sure you're `use`-ing the correct database (ex: `use twitter_sample`), and then execute the following from the mongo shell:

```bash
db.tweets.createIndex(
   { id: 1 },
   { name: "id_index", unique: true }
)
```

You can run `db.tweets.getIndexes()` to check that the index was created successfully.

### Execute load scripts

Insert one vs. bulk insert





In [2]:
import Clean_Load_Scripts as cleanNLoad

In [3]:
data_folder = '/Users/linkalis/Desktop/twitter_data/twitter_sample_5GB_split/'
logs_folder = '/Users/linkalis/Desktop/twitter_data/twitter_sample_5GB_split/logs/'

In [4]:
extractor = cleanNLoad.Extractor(data_folder, logs_folder, initialize=True)

In [ ]:
while extractor.next_file_available():
    next_file_data, next_file_name = extractor.get_next_file() # read in the next file
    cleaner = cleanNLoad.Cleaner(next_file_data, next_file_name, logs_folder) # clean the data (fix bounding boxes, add centroids, etc.)
    cleaned_data = cleaner.clean_data() 
    loader = cleanNLoad.Loader(cleaned_data, next_file_name, logs_folder) # initialize the loader
    loader.get_connection("mongodb", "localhost", "27017", None, None, "twitter_sample", "tweets") # create a database connection
    loader.load_batch_data() # load the file's data as a batch

<hr>

## Query the data

### Basic queries

Now that the data is loaded, let's look at how to perform a few basic queries.  MongoDB uses special query syntax that resembles the syntax used in JavaScript/JSON objects.  If you are familiar with JavaScript, you will likely recognize the parentheses `()` and brackets `{}` that make up the backbone of MongoDB's query structure.  A basic query is formulated as follows:

```
{ 'attribute_name.sub_attribute_name': { $comparison_operator: 'value' } }
```

You can access attributes of the document and their sub-attributes using selection syntax that is similar to JavaScript.  For example, tweets contain an attribute called 'user', and each user contains a sub-attribute called 'followers_count'.  So, to query information about users' follower counts, you would search on the field: `'user.follower_count'`.

MongoDB query syntax also includes some special [comparison operators](https://docs.mongodb.com/manual/reference/operator/query-comparison/) that you can use within a query:

* $eq: equals

* $gt: greater than

* $gte: greater than or equal to

* $in: value is within a specified array

* $lt: less than

* $lte: less than or equal to

* $ne: not equal to

* $nin: not in

Now, let's look at some sample questions and how you can query for answers in MongoDB.  To help you get started, the Mongo Compass app has a "Filter" interface that lets you write simple queries and return the results directly within the app.  To run the queries below, simply paste them into your Mongo Compass app's "Filter" box and hit "Enter" to return the results.

![Mongo Compass filter input box](img_mongodb/mongodb_filter_box.png)

#### Query: Which tweets were tweeted by users who have 5,000 or more followers?

```
{ 'user.followers_count': {$gte: 5000} }
```

When you run this query, you may notice that it take a substantial amount of time to run.  To understand why this is, it helps to look at the **explain plan** for the query.  Click into the "Explain Plan" tab in the Mongo Compass app, then run the query again to get information on how the query is being executed behind the scenes:

![Mongo Compass explain plan without index](img_mongodb/mongodb_explain_plan_no_index.png)

If to look at the "Documents Examined" information, it looks like the query has to examine _every single document_ in the database before it is able to return a result!  Why?  Well, the warning icon at the end of the "Query Performance Summary" box gives you a hint: there is "no index available" that helps this query run faster.  Without an index on the followers_count values, the database has to search through every individual tweet document to determine whether the followers_count value is greater than or equal to 5,000.  The followers_count values are not sorted in any way, so there is no shortcut available to.  To fix this, simply add an index on the followers_count field.  It takes a while to generate the index, and the index itself takes up some additional space on disk, but it will vastly improve the response time for this query:

<img src="img_mongodb/mongodb_followers_count_index.png" alt="Mongo Compass followers count index" style="width: 400px;"/>

Once the index is ready, execute the query again and it should return results much faster.  You can see that the database first conducts an index scan ("IXSCAN") to split the tweets into those that were tweeted by users with >= 5,000 followers, and those that were tweeted by users with < 5,000 followers.  This process only takes a few hundred milliseconds.  Then, the database performs a fetch operation ("FETCH") to actually read the tweets and return them to the results list.  This fetch process takes substantially longer than the index scan operation:

![Mongo Compass explain plan with index](img_mongodb/mongodb_explain_plan_with_index.png)

This means that, with an index in place, the database is able to sort through the tweets extremely fast, and the time it takes to return results is largely dependent on the number of results being returned in the fetch operation.  (To convince yourself of this, try substituting a followers_count of `100000`, rather than `5000` and see the difference in performance that occurs when the number of results being returned is much lower.)

This example illustrates an important tradeoff in MongoDB and document-based databases.  The data load process is often very easy and quick in document-based databases, but the data itself is relatively unstructured behind the scenes.  Without a specific pre-defined data structure--some kind of a data model or indexed table working behind the scenes--it can be tricky to query data efficiently without some additional effort.  It is common, then, to have to generate indexes to help sort the documents based on some key attributes of interest before performing queries within a document-based database.

### Spatial queries

This time around, instead of using the Mongo Compass app, we'll use the mongo shell to query and return results.  This is because Mongo's spatial query syntax can be somewhat complicated and can require multiple lines of code, and the mongo shell is a slightly more convenient interface to use when you're writing complex, multi-line queries.  To run the queries below, simply paste them into your mongo shell and hit "Enter" to return the results.


Add spatial indexes:
    
https://docs.mongodb.com/manual/core/geohaystack/

### Advanced queries

#### Query: How many tweets mention coffee and are associated with Minnesota?

This query is pretty general in purpose, so it would be nice if we could conduct a very general text search to address this query.  Fortunately, MongoDB allows you to set up a **text index** that enables full-text search across multiple fields at once.  The [MongoDB documentation on text search](https://docs.mongodb.com/manual/text-search/) explains: "a collection can only have one text search index, but that index can cover multiple fields."  So, let's set up an index that combines all of the important open-ended text fields in the data: the tweet 'text' field, the 'user.description' field and the 'place.full_name' field.  Run the following from the mongo shell to create the index.  This may take a minute or two to complete:

```
db.tweets.createIndex({ 'text': "text", 'user.description': "text", 'place.full_name': "text" })
```

Then, simply conduct a text search query for the words "minnesota" and "coffee" to find tweets that contain both words somewhere in their open-ended text fields.  Then, sort the tweets by relevance score and show the top 10:

```
db.tweets.find(
   { $text: { $search: "\"minnesota\" \"coffee\""} },
   { text: 1, 'user.description': 1, 'place.full_name': 1, score: { $meta: "textScore" } }
).sort( { score: { $meta: "textScore" } } ).limit(10)
```

Notice the escape syntax (ex: `\"minnesota\"`) in the search string above.  The `\"` surrounding each search term is what tells MongoDB to conduct a search that requires both Minnesota AND coffee to be in the result.  If you want to search for Minnesota OR coffee, you can simply use `"minnesota coffee"` as your search string, and the search will return results that have one term or the other, but not necessarily both.

One important limitation of text indexes like this one is that they acts as "special indexes" that cannot be combined with complex spatial queries that require their own special indexes.  The MongoDB documentation on the [text query operator](https://docs.mongodb.com/manual/reference/operator/query/text/) explains: "You cannot combine the \$text expression, which requires a special text index, with a query operator that requires a different type of special index. For example you cannot combine \$text expression with the \$near operator."

#### Query: Which countries have the most tweets associated with them?

For this type of query, MongoDB offers a syntax structure called the **aggregation pipeline** that is well suited to queries that involve matching/finding values, grouping these values, and then aggregating them in some way.  The documentation on [aggregation pipelines](https://docs.mongodb.com/manual/core/aggregation-pipeline/) explains how to structure an aggregation-style query, so let's examine how to put this into action.

First, we'll want to add an index on the `'place.place_type'` and `'place.id'` values.  This may take a minute or two to complete:

```
db.tweets.createIndex({ 'place.place_type': 1, 'place.id': 1 })
```

Next, formulate the aggregate pipeline.  The query below is structured as follows: 

1) Look for ("match") all places with a place_type of 'city'. 

2) Group by the `place.id` and `place.full_name` fields to use as the id field in the results summary; rename these field 'place_id' and 'place_full_name', which is what will show up in the summary.  Also create a summary field called tweet_count.  Set up this field so that it keeps a running sum of how many tweets associated with each city, adding the value '1' each time a tweet is detected for a particular place.

3) Sort the results in descending order, so the places with the greatest number of tweets rise to the top.

4) Show the top 10 results.

```
db.tweets.aggregate([
    { $match: { 'place.place_type': 'city' } },
    { $group: { _id: { place_id: '$place.id', place_full_name: '$place.full_name' }, tweet_count: { $sum: 1 } } },
    { $sort : { tweet_count : -1 } },
    { $limit: 10 }
]).explain("executionStats")
```

The result is an easy-to-read summary table, with place_id, place_full_name, and tweet_count fields present.  Are you surprised by any of the results?

#### Query: Which cities have the most tweets associated with them?

In additional to the aggregation pipeline syntax, MongoDB also offers a [map-reduce syntax](https://docs.mongodb.com/manual/core/map-reduce/) that offers similar aggregation functionality, but that may prove slightly more flexible for complex queries and larger datasets.  Let's look at a simple example here.  The query below is structured as follows:

1. Define the **map function**.  The first function that is passed as an argument to MongoDB's mapReduce method is the function that will be applied to each document.  Its goal is to emit key-value pairs that are passed to the `reduce` function for further processing.  In this example, since we only want a simple count of the number of tweets, we only need to pass the place.full_name as the key field, and the value '1' as the value field.

2. Define the **reduce function**.  This second function takes all of the key-value pairs emitted in the first phase and combines them using whatever aggregate function we want to use.  Here, it is outputting a simple sum of the '1' values to keep a running total of the number of tweets for each place.

3. Define the `query` filters.  Here, we stipulate that the `place.place_type` must be equal to 'city'.

4. Define the `out` option, which is the name to use for the collection that the results will be written to.

```
db.tweets.mapReduce(
    function() { emit(this.place.full_name, 1); },
    function(key, val) { return Array.sum(val); },
    {
        query: { 'place.place_type': 'city' },
        out: 'tweet_count'
    }
)
```

De